In [1]:
from __future__ import division
import os
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imsave, show
from IPython.display import Image, HTML
from keras.models import load_model
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.


In [2]:
from constants import LABELS, WEATHER_IDX
from utils import F2Score, get_predictions
from datasets.weather_in_input import WeatherInInputDataset
from models.gui import GuiNet

## Load data

In [3]:
# Loads dataset and model
with open("./backups/model_with_weather_input/training-files.csv") as f_training_files, open("./backups/model_with_weather_input/validation-files.csv") as f_validation_files:
    training_files = f_training_files.readline().split(",")
    validation_files = f_validation_files.readline().split(",")

data = WeatherInInputDataset([], training_files=training_files, validation_files=validation_files)
model = GuiNet(data, model=load_model("./backups/model_with_weather_input/checkpoint.hdf5"), n_gpus=0)

In [6]:
(x_validate, y_validate) = data.validationSet(model.image_data_fmt, model.input_shape)
rawPredictions = []

def scoring(model, inputs, expectations):
    global rawPredictions
    rawPredictions = model.model.predict(inputs, verbose=1, batch_size=16)
    predictions = get_predictions(rawPredictions, model.data.labels)
    predictions = np.array([x for x in predictions])
    return np.mean([F2Score(
        prediction,
        [LABELS[i] for i, x in enumerate(expectation) if x == 1 and i in model.data.label_idx]
    ) for prediction, expectation in zip(predictions, expectations)])
            
score = scoring(model, x_validate, y_validate)
print("\nscore", score)

4048/4048 [==============================] - 17s    

score 0.946164218695


In [7]:
expected_tags = {}
all_tags = set(LABELS)

with open("./rawInput/train.csv", "r") as train_labels:
    train_labels.readline()
    for line in train_labels:
        img, tags = line.rstrip("\n").split(",")
        expected_tags[img] = tags.split(" ")
print(len(expected_tags))

40479


In [8]:
predicted_tags = {}
with open("./backups/model_with_weather_input/train-predict.csv", "r") as predict_labels:
    predict_labels.readline()
    for line in predict_labels:
        img, tags = line.rstrip("\n").split(",")
        predicted_tags[img] = tags.split(" ")
print(len(predicted_tags))

40479


In [9]:
predicted_proba = {}
with open("./backups/model_with_weather_input/train-predict-raw.csv", "r") as predict_proba:
    tags = predict_proba.readline().rstrip("\n").split(",")[1].split(" ")
    i = 0
    for line in predict_proba:
        img, proba = line.rstrip("\n").split(",")
        proba = proba.split(" ")
        predicted_proba[img] = { t: float(v) for t, v in zip(tags, proba) }
        predicted_tags[img] = [tag for tag in predicted_proba[img] if predicted_proba[img][tag] > 0.2]
#         print(proba, rawPredictions[i])
        i += 1
print(len(predicted_proba))

40479


In [10]:
training_set = set([])
validation_set = set([])
with open("./backups/model_with_weather_input/training-files.csv", "r") as training_files:
    original_training_set = set(training_files.readline().strip().split(","))
    for f in os.listdir("./rawInput/train-jpg"):
        filename = f.split(".")[0]
        s = None
        if filename in original_training_set:
            s = training_set
        elif filename in predicted_tags:
            s = validation_set
        if s is not None:
            s.add(filename)
#         for i in range(8):
#             s.add("{name}--{i}".format(name=filename, i=i))

print("total images:", len(os.listdir("./rawInput/train-jpg")))
print("training set:", len(training_set))
print("validation set:", len(validation_set))

total images: 40479
training set: 36431
validation set: 4048


## Read predictions

### Per label score

In [11]:
# First value is number of correctly predicted
# Second is number of not detected (false negative)
# Third is number of inaccurately predicted (false positive)
tags_score = {t: { "tp":0, "fn": 0, "fp": 0} for t in all_tags}

def scores(tp, fp, fn):
    p = tp / max(1, (tp + fp))
    r = tp / max(1, (tp + fn))
    b = 2
    if p == 0 or r == 0:
        f2 = 0
    else:
        b = 2
        f2 = (1 + b**2) * p * r / (b**2*p + r)
    return p, r, f2
    
for img in validation_set:
    true_tags = expected_tags[img]
    pred_tags = predicted_tags[img]
    for t in true_tags:
        if t in pred_tags:
            tags_score[t]["tp"] += 1
        else:
            tags_score[t]["fn"] += 1
    for t in pred_tags:
        if t not in true_tags:
            tags_score[t]["fp"] += 1

data = []
for tag in all_tags:
    s = tags_score[tag]
    p, r, f2 = scores(s["tp"], s["fp"], s["fn"])
    data.append([tag, s["tp"], s["fp"], s["fn"], p, r, f2])
data.sort(key = lambda l:l[-1])
data = np.array(data)
columns = ["tp", "fp", "fn", "validation precision", "validation recall", "f2"]
df = pd.DataFrame(data=data[:, 1:], index=data[:, 0], columns=columns)
df

,tp,fp,fn,validation precision,validation recall,f2
blooming,0,5,32,0.0,0.0,0
conventional_mine,0,0,15,0.0,0.0,0
slash_burn,0,2,19,0.0,0.0,0
blow_down,1,0,8,1.0,0.1111111111111111,0.13513513513513514
bare_ground,13,35,69,0.2708333333333333,0.15853658536585366,0.17287234042553193
selective_logging,15,29,17,0.3409090909090909,0.46875,0.436046511627907
cultivation,298,419,99,0.41562064156206413,0.7506297229219143,0.6464208242950108
habitation,315,189,73,0.625,0.8118556701030928,0.7660505836575877
artisinal_mine,23,6,7,0.7931034482758621,0.7666666666666667,0.7718120805369127
water,613,352,112,0.6352331606217616,0.8455172413793104,0.7930142302716688


## F2 score

In [14]:
def scoreSet(data_set):
    print("scoreSet", len(data_set), len(expected_tags), len(predicted_tags))
    scores = []
    f2, n = 0, 0
    for img in data_set:
        real_tags = expected_tags[img]
        pred_tags = predicted_tags[img]
        s = F2Score(pred_tags, real_tags)
        f2 += s
        scores.append({ "img": img, "f2": s })
        n += 1
    return f2 / n, scores

In [15]:
training_score, training_scores = scoreSet(training_set)
validation_score, validation_scores = scoreSet(validation_set)
training_scores.sort(key = lambda l: l["f2"])
validation_scores.sort(key = lambda l: l["f2"])

print("F2 score for training set", training_score)
print("F2 score validation set", validation_score)

scoreSet 36431 40479 40479
scoreSet 4048 40479 40479
F2 score for training set 0.9445018506402579
F2 score validation set 0.9461642186954404


## Prediction per image

In [17]:
short_labels = {"agriculture": "ag","artisinal_mine": "a_mi","bare_ground": "b_gr","blooming": "bloom","blow_down": "blow","clear": "clea","cloudy": "clou","conventional_mine": "c_mi","cultivation": "cult","habitation": "hab","haze": "haze","partly_cloudy": "p_clo","primary": "prim","road": "road","selective_logging": "s_log","slash_burn": "sl_b","water": "wat"}

# return an HTML display for one prediction entry, making it green, red or grey
def getLabelPred(tag, img):
    truth = tag in expected_tags[img]
    pred = tag in predicted_tags[img]
    prob = predicted_proba[img][tag]
    isGood = truth == pred
    if truth == pred:
        cl = "green" if truth else "grey"
    else:
        cl = "red true" if truth else "red false"
    return '<span title="{tag}" class="{cl}">{val}</span>'.format(tag=tag, val=round(prob, 2), cl=cl)
        
# Return a dataframe the predictions
def getPrediction(data):
    img = data["img"]
    f2 = data["f2"]
    image = '<img src="./rawInput/train-jpg/{img}.jpg" class="input-image"/>'.format(img=img)
    
    return [
        img, 
        image, 
        f2
    ] + [getLabelPred(t, img) for t in all_tags]

table = []
originals = set([])
for data in validation_scores[:1000]:
    img = data["img"]
    original = img.split("--")[0]
    if original not in originals:
        table.append(getPrediction(data))
        originals.add(original)

table = np.array(table)

columns = ["image", "f2"] + [short_labels[t] for t in all_tags]
df = pd.DataFrame(data=table[:, 1:], index=table[:, 0], columns=columns)
style = "<style>.input-image{width: 50px; max-width: 250px !important;} .input-image:hover{width: 250px;} .green{color: green;} .grey{color: #e0e0e0;} .red{color: black;} .true{border-bottom: 3px solid green;} .false{border-bottom: 3px solid red;}</style>"
HTML(style + df.to_html(escape=False))

,image,f2,road,bloom,clou,haze,wat,a_mi,clea,prim,p_clo,sl_b,c_mi,hab,s_log,b_gr,ag,cult,blow
train_2012,,0.22727272727272727,0.25,0.0,0.0,0.0,0.58,0.01,1.0,0.96,0.0,0.01,0.0,0.19,0.01,0.08,0.75,0.15,0.0
train_35377,,0.2777777777777778,0.08,0.0,0.0,0.0,0.57,0.0,0.0,0.91,1.0,0.0,0.0,0.01,0.0,0.01,0.76,0.04,0.0
train_18337,,0.2777777777777778,0.07,0.0,0.0,0.0,0.23,0.0,1.0,0.54,0.0,0.0,0.0,0.02,0.0,0.14,0.72,0.03,0.0
train_20477,,0.2777777777777778,0.09,0.0,0.0,0.0,0.74,0.0,1.0,0.79,0.0,0.0,0.0,0.01,0.0,0.17,0.59,0.05,0.0
train_17851,,0.3225806451612903,0.55,0.0,0.0,0.0,0.21,0.0,1.0,1.0,0.0,0.01,0.0,0.38,0.01,0.04,0.89,0.25,0.0
train_27496,,0.3225806451612903,0.39,0.04,0.0,0.0,0.14,0.02,1.0,0.99,0.0,0.01,0.0,0.38,0.23,0.04,0.36,0.22,0.01
train_26402,,0.3571428571428571,0.04,0.0,0.0,0.0,0.14,0.0,1.0,0.72,0.0,0.0,0.0,0.02,0.0,0.09,0.65,0.03,0.0
train_17657,,0.3703703703703703,0.84,0.0,0.0,0.0,0.19,0.01,1.0,0.71,0.0,0.0,0.09,0.2,0.0,0.35,0.68,0.04,0.0
train_13190,,0.3846153846153846,0.22,0.01,0.0,0.0,0.32,0.0,1.0,1.0,0.0,0.01,0.0,0.04,0.01,0.03,0.77,0.31,0.0
train_24586,,0.3846153846153846,0.3,0.0,0.0,1.0,0.57,0.0,0.0,0.95,0.0,0.0,0.0,0.23,0.0,0.01,0.43,0.08,0.0
